In [1]:
# imports
import matplotlib.pyplot as plt
import numpy as np

from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.circuit.library.data_preparation.pauli_feature_map import PauliFeatureMap
from qiskit.circuit import Parameter, ParameterVector

# Qiskit Implementation

In [2]:
# Qiskit Z-Z Feature Map

from typing import Callable, List, Union, Optional

class ZZFeatureMap(PauliFeatureMap):
    """Second-order Pauli-Z evolution circuit.

    For 3 qubits and 1 repetition and linear entanglement the circuit is represented by:

    .. parsed-literal::

        ┌───┐┌─────────────────┐
        ┤ H ├┤ U1(2.0*φ(x[0])) ├──■────────────────────────────■────────────────────────────────────
        ├───┤├─────────────────┤┌─┴─┐┌──────────────────────┐┌─┴─┐
        ┤ H ├┤ U1(2.0*φ(x[1])) ├┤ X ├┤ U1(2.0*φ(x[0],x[1])) ├┤ X ├──■────────────────────────────■──
        ├───┤├─────────────────┤└───┘└──────────────────────┘└───┘┌─┴─┐┌──────────────────────┐┌─┴─┐
        ┤ H ├┤ U1(2.0*φ(x[2])) ├──────────────────────────────────┤ X ├┤ U1(2.0*φ(x[1],x[2])) ├┤ X ├
        └───┘└─────────────────┘                                  └───┘└──────────────────────┘└───┘

    where ``φ`` is a classical non-linear function, which defaults to ``φ(x) = x`` if and
    ``φ(x,y) = (pi - x)(pi - y)``.

    Examples:

        >>> prep = ZZFeatureMap(2, reps=2)
        >>> print(prep)
             ┌───┐┌──────────────┐
        q_0: ┤ H ├┤ U1(2.0*x[0]) ├──■───────────────────────────────────────■──
             ├───┤├──────────────┤┌─┴─┐┌─────────────────────────────────┐┌─┴─┐
        q_1: ┤ H ├┤ U1(2.0*x[1]) ├┤ X ├┤ U1(2.0*(pi - x[0])*(pi - x[1])) ├┤ X ├
             └───┘└──────────────┘└───┘└─────────────────────────────────┘└───┘

        >>> from qiskit.circuit.library import EfficientSU2
        >>> classifier = ZZFeatureMap(3) + EfficientSU2(3)
        >>> classifier.num_parameters
        15
        >>> classifier.parameters  # 'x' for the data preparation, 'θ' for the SU2 parameters
        {Parameter(θ[9]), Parameter(θ[4]), Parameter(θ[6]), Parameter(θ[1]), Parameter(x[2]),
        Parameter(θ[7]), Parameter(x[1]), Parameter(θ[8]), Parameter(θ[2]), Parameter(θ[10]),
        Parameter(θ[5]), Parameter(θ[0]), Parameter(θ[3]), Parameter(x[0]), Parameter(θ[11])}
        >>> classifier.count_ops()
        OrderedDict([('u1', 12), ('cx', 12), ('ry', 12), ('cz', 9), ('h', 6)])
    """

    def __init__(
        self,
        feature_dimension: int,
        reps: int = 2,
        entanglement: Union[str, List[List[int]], Callable[[int], List[int]]] = "full",
        data_map_func: Optional[Callable[[np.ndarray], float]] = None,
        insert_barriers: bool = False,
        name: str = "ZZFeatureMap",
    ) -> None:
        """Create a new second-order Pauli-Z expansion.

        Args:
            feature_dimension: Number of features.
            reps: The number of repeated circuits, has a min. value of 1.
            entanglement: Specifies the entanglement structure. Refer to
                :class:`~qiskit.circuit.library.NLocal` for detail.
            data_map_func: A mapping function for data x.
            insert_barriers: If True, barriers are inserted in between the evolution instructions
                and hadamard layers.

        Raises:
            ValueError: If the feature dimension is smaller than 2.
        """
        if feature_dimension < 2:
            raise ValueError(
                "The ZZFeatureMap contains 2-local interactions and cannot be "
                f"defined for less than 2 qubits. You provided {feature_dimension}."
            )

        super().__init__(
            feature_dimension=feature_dimension,
            reps=reps,
            entanglement=entanglement,
            paulis=["Z", "ZZ"],
            data_map_func=data_map_func,
            insert_barriers=insert_barriers,
            name=name,
        )

In [3]:
# Show the feature map for a 3 qubit system, with 1 repetition
prep = ZZFeatureMap(3, reps=1)
print(prep)

     ┌───┐┌──────────────┐                                                »
q_0: ┤ H ├┤ U1(2.0*x[0]) ├──■─────────────────────────────────────■────■──»
     ├───┤├──────────────┤┌─┴─┐┌───────────────────────────────┐┌─┴─┐  │  »
q_1: ┤ H ├┤ U1(2.0*x[1]) ├┤ X ├┤ U1(2.0*(π - x[0])*(π - x[1])) ├┤ X ├──┼──»
     ├───┤├──────────────┤└───┘└───────────────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ U1(2.0*x[2]) ├───────────────────────────────────────────┤ X ├»
     └───┘└──────────────┘                                           └───┘»
«                                                »
«q_0: ───────────────────────────────────■───────»
«                                        │       »
«q_1: ───────────────────────────────────┼────■──»
«     ┌───────────────────────────────┐┌─┴─┐┌─┴─┐»
«q_2: ┤ U1(2.0*(π - x[0])*(π - x[2])) ├┤ X ├┤ X ├»
«     └───────────────────────────────┘└───┘└───┘»
«                                           
«q_0: ──────────────────────────────────────
«                    

C:\Users\julia\anaconda3\lib\site-packages\sympy\core\expr.py:2451: SymPyDeprecationWarning: 

expr_free_symbols method has been deprecated since SymPy 1.9. See
https://github.com/sympy/sympy/issues/21494 for more info.

  SymPyDeprecationWarning(feature="expr_free_symbols method",


# Custom Implementation

In [4]:
# Define custom function to take the number of qubits and desired repetitions
# as input, and return the ZZ-feature mapping

def myZZ(numQ, numReps):
    qc = QuantumCircuit(numQ)
    parameters = ParameterVector('x', numQ)
    for i in range(numReps):
        # Apply H gate and Rz to each qubit
        for j in range(numQ):
            qc.h(j)
        for j in range(numQ):
            qc.rz(2.0*(parameters[j]),j)
        # Linear entanglement and Rz    
        for j in range(numQ-1):
            k = 0
            while(j+k < numQ-1):
                qc.cx(j,j+k+1)
                qc.rz(2.0*((np.pi-parameters[j])*(np.pi-parameters[j+k+1])),j+k+1)
                qc.cx(j,j+k+1)
                k=k+1
    print(qc.draw())

In [5]:
# Test for system with 3 qubits and 1 repetition
myZZ(3,1)

     ┌───┐┌──────────────┐                                                »
q_0: ┤ H ├┤ Rz(2.0*x[0]) ├──■─────────────────────────────────────■────■──»
     ├───┤├──────────────┤┌─┴─┐┌───────────────────────────────┐┌─┴─┐  │  »
q_1: ┤ H ├┤ Rz(2.0*x[1]) ├┤ X ├┤ Rz(2.0*(π - x[0])*(π - x[1])) ├┤ X ├──┼──»
     ├───┤├──────────────┤└───┘└───────────────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ Rz(2.0*x[2]) ├───────────────────────────────────────────┤ X ├»
     └───┘└──────────────┘                                           └───┘»
«                                                »
«q_0: ───────────────────────────────────■───────»
«                                        │       »
«q_1: ───────────────────────────────────┼────■──»
«     ┌───────────────────────────────┐┌─┴─┐┌─┴─┐»
«q_2: ┤ Rz(2.0*(π - x[0])*(π - x[2])) ├┤ X ├┤ X ├»
«     └───────────────────────────────┘└───┘└───┘»
«                                           
«q_0: ──────────────────────────────────────
«                    